## Questão 3

In [ ]:
# Imports essenciais

import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from ucimlrepo import fetch_ucirepo
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

In [11]:

# 1) Carregar a base de dados
cdc = fetch_ucirepo(id=891)
X_all = cdc.data.features
y = cdc.data.targets.squeeze().rename('Diabetes_binary')

# 2) Combinar features e target num DataFrame
df = pd.concat([X_all, y], axis=1)

# 3) Identificar variável-alvo
#    - “Diabetes_binary”: 0 = não diagnosticado, 1 = diagnosticado
print(df['Diabetes_binary'].value_counts())

# 4) Selecionar variáveis explicativas
#    Conforme analise previa em Q1
features = ['GenHlth', 'HighBP', 'HighChol', 'BMI', 'PhysActivity']
X = df[features]

# 5) Justificativa concisa
#    - GenHlth e HighBP foram as top correlacionadas com o target (|r| ≈ 0.30 e 0.20).  
#    - HighChol e BMI apresentam forte evidência clínica e correlação moderada.  
#    - PhysActivity capturou padrão não-linear relevante via informação mútua.  
#   Essas 5 variáveis formam um conjunto enxuto e informativo, otimizado para inferir risco de diabetes.

# Questão 2 B
# Supondo X e y já definidos:
# X: DataFrame com as features selecionadas
# y: Série com a variável-alvo 'Diabetes_binary'

# Realiza a separação em treino e teste de forma estratificada
X_train, X_test, y_train, y_test = train_test_split(
    X,                # Features
    y,                # Target
    test_size=0.2,    # 20% dos dados para teste
    random_state=42,  # garante reprodutibilidade
    stratify=y        # preserva a proporção de classes em ambos conjuntos
)

# Verificação das proporções originais e pós-split
print("Distribuição original:", y.value_counts(normalize=True).to_dict())
print("Distribuição treino:  ", y_train.value_counts(normalize=True).to_dict())
print("Distribuição teste:   ", y_test.value_counts(normalize=True).to_dict())


#Questão 2D Otimização do KNN

# 2) Pipeline de pré-processamento + KNN
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier())
])

# 3) Grade de hiperparâmetros incluindo weighting
param_grid = {
    'knn__n_neighbors': list(range(1, 31, 2)),   # valores ímpares de 1 a 29
    'knn__weights': ['uniform', 'distance'],     # uniform vs distance weighted
    'knn__metric': ['minkowski', 'euclidean']    # distância padrão
}

# 4) GridSearchCV estratificado, otimizando F1 da classe positiva
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid = GridSearchCV(
    pipe,
    param_grid,
    scoring='f1',
    cv=cv,
    n_jobs=-1,
    verbose=1,
    refit=True
)

# 5) Ajuste no conjunto de treino
grid.fit(X_train, y_train)

# 6) Melhor combinação
best_params = grid.best_params_
best_score = grid.best_score_
print(f"Melhores parâmetros: {best_params}")
print(f"Melhor F1 CV (classe1): {best_score:.4f}\n")

# 7) Avaliação final no teste
y_pred = grid.predict(X_test)
print("Relatório de Classificação (Conjunto de Teste):")
print(classification_report(y_test, y_pred, digits=4))


Diabetes_binary
0    218334
1     35346
Name: count, dtype: int64
Distribuição original: {0: 0.8606669820245979, 1: 0.13933301797540207}
Distribuição treino:   {0: 0.8606659965310628, 1: 0.13933400346893723}
Distribuição teste:    {0: 0.8606709239987386, 1: 0.13932907600126143}
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Melhores parâmetros: {'knn__metric': 'minkowski', 'knn__n_neighbors': 1, 'knn__weights': 'uniform'}
Melhor F1 CV (classe1): 0.2733

Relatório de Classificação (Conjunto de Teste):
              precision    recall  f1-score   support

           0     0.8843    0.8929    0.8886     43667
           1     0.2962    0.2785    0.2871      7069

    accuracy                         0.8073     50736
   macro avg     0.5903    0.5857    0.5878     50736
weighted avg     0.8024    0.8073    0.8048     50736



In [12]:
# Questão 3 A) Validação Cruzada Estratificada do KNN


from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.neighbors import KNeighborsClassifier

# Pipeline que aplica StandardScaler e o KNN com os hiperparâmetros ótimos
pipeline_knn = Pipeline([
    ('scaler', StandardScaler()),  
    ('knn', KNeighborsClassifier(
        n_neighbors=1,         # melhor K encontrado
        weights='uniform',     # melhor policy de weighting
        metric='minkowski'     # melhor métrica de distância
    ))
])

# Estratégia de validação cruzada estratificada (5 folds)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Executa cross-validation no conjunto de treino original
f1_scores = cross_val_score(
    pipeline_knn,
    X_train,         # features de treino não escalonadas
    y_train,         # target de treino
    cv=cv,
    scoring='f1',    # otimiza F1-score da classe positiva
    n_jobs=-1        # paraleliza em todos os núcleos disponíveis
)

# Exibe resultados
print("F1-score por fold:", np.round(f1_scores, 4))
print("Média F1 (CV):      ", np.round(f1_scores.mean(), 4))
print("Desvio Padrão (CV): ", np.round(f1_scores.std(), 4))


F1-score por fold: [0.2696 0.2656 0.2741 0.2875 0.2696]
Média F1 (CV):       0.2733
Desvio Padrão (CV):  0.0076


## Como a Validação Cruzada Estima o Desempenho Real do Modelo

1. **Uso Eficiente de Dados**  
   - Em vez de depender de uma única divisão treino/teste, a validação cruzada (CV) utiliza todo o conjunto de treinamento em múltiplos cenários: cada porção dos dados é, em algum momento, reservada para validação.  
   - Isso maximiza o aproveitamento do dataset, especialmente importante em problemas desbalanceados ou com poucos exemplos da classe minoritária.

2. **Redução de Variância na Estimativa**  
   - Ao calcular o F1-score em **5 folds**, obtemos cinco estimativas independentes de desempenho.  
   - A **média** dos F1-scores representa uma projeção mais estável do que um único teste, enquanto o **desvio padrão** quantifica a consistência do modelo ao longo de diferentes subconjuntos.

3. **Detecção de Overfitting**  
   - Se o modelo estiver muito ajustado aos dados de treino, ele terá desempenho significativamente pior em algumas folds, elevando o desvio padrão.  
   - Um baixo desvio padrão (≈ 0.0076 aqui) indica que o modelo generaliza de forma similar em diferentes amostras, sugerindo menor risco de overfitting.

4. **Avaliação Realista**  
   - A CV reflete variações naturais da população amostrada: capturar essa heterogeneidade é crucial para prever como o modelo se comportará em novos dados não vistos.  
   - Especialmente em triagem de saúde, onde cada falso negativo pode ter alto custo, a CV fornece uma visão mais confiável da sensibilidade (recall) e do equilíbrio entre precision e recall (F1).

5. **Base para Comparação**  
   - Quando compararmos algoritmos ou ajustes de hiperparâmetros, o F1 médio em CV serve como critério **robusto e isento de viés** para escolher a melhor configuração, antes de validá‐la no conjunto de teste final.

> **Resumo:**  
> A validação cruzada transforma múltiplas divisões de treino/teste em um “termômetro” mais preciso da performance real, garantindo que o modelo KNN escolhido (K = 1, F1 médio ≈ 0.2733) seja avaliado de forma estável, representativa e confiável para a triagem de diabetes.  


In [14]:
#Questao 3 B 

#Validação Cruzada Estratificada com Regressão Logística

import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression

# 1) Construir pipeline com normalização e Regressão Logística
pipeline_logistic = Pipeline([
    ('scaler', StandardScaler()),  
    ('logreg', LogisticRegression(
        class_weight='balanced',  # corrige desequilíbrio de classes
        solver='lbfgs',
        max_iter=1000,
        random_state=42
    ))
])

# 2) Configurar validação cruzada estratificada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 3) Executar cross-validation para F1-score da classe positiva
f1_scores_logreg = cross_val_score(
    pipeline_logistic,
    X_train,    # features de treino originais (não escalonadas)
    y_train,    # target de treino
    cv=cv,
    scoring='f1',  # F1 da classe 1
    n_jobs=-1      # paraleliza em todos os núcleos disponíveis
)

# 4) Exibir resultados
print("=== Regressão Logística (CV Estratificado) ===")
print("F1-score por fold:", np.round(f1_scores_logreg, 4))
print("Média F1 (CV):      ", np.round(f1_scores_logreg.mean(), 4))
print("Desvio Padrão (CV): ", np.round(f1_scores_logreg.std(), 4))


=== Regressão Logística (CV Estratificado) ===
F1-score por fold: [0.4225 0.4255 0.4286 0.427  0.4252]
Média F1 (CV):       0.4258
Desvio Padrão (CV):  0.002


## Avaliação da Regressão Logística em CV Estratificado

- **F1‐scores por fold:** [0.4225, 0.4255, 0.4286, 0.4270, 0.4252]  
- **Média F1 (CV):** 0.4258  
- **Desvio Padrão (CV):** 0.0020  

### Principais Observações

1. **Performance Superior ao KNN**  
   - A Regressão Logística alcançou F1 médio ≈ 0.426, bem acima dos ≈ 0.273 do KNN otimizado.  
   - Isso indica que um modelo linear penalizado (com `class_weight='balanced'`) consegue capturar melhor os padrões de diabetes nas 5 features selecionadas.

2. **Alta Consistência (Baixo Desvio Padrão)**  
   - O desvio de ≈ 0.002 reflete **muito pouca variabilidade** entre as folds, mostrando que o modelo generaliza de forma estável em diferentes subconjuntos estratificados.

3. **Impacto do `class_weight='balanced'`**  
   - Ajustar os pesos na regressão compensou o desbalanceamento (~86/14), aumentando recall da classe positiva sem sacrificar excessivamente a precision, resultando em um F1 robusto.

4. **Trade-off Viés-Variância**  
   - A Regressão Logística, sendo menos flexível que o KNN com K=1, evita overfitting e entrega desempenho mais confiável e interpretável.

---

> **Conclusão Parcial:**  
> A Regressão Logística serve como um modelo de **baseline forte**, combinando simplicidade, estabilidade e boa capacidade de detecção de casos de diabetes (F1 ≈ 0.426). Em comparação, o KNN mostrou-se menos eficaz e mais variável, justificando a adoção da Regressão Logística como ponto de partida antes de testar modelos mais complexos ou técnicas de ensemble.  


In [15]:
# Questão 3B) Validação Cruzada Estratificada com Árvore de Decisão 

import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score

# 1) Pipeline com Árvore de Decisão e balanceamento de classes
pipeline_tree = Pipeline([
    ('clf', DecisionTreeClassifier(
        class_weight='balanced', 
        random_state=42
    ))
])

# 2) Estratégia de validação cruzada estratificada (5 folds)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 3) Executar cross-validation usando F1-score da classe positiva
f1_scores_tree = cross_val_score(
    pipeline_tree,
    X_train,    # features de treino originais
    y_train,    # target de treino
    cv=cv,
    scoring='f1',  
    n_jobs=-1      
)

# 4) Exibir resultados
print("=== Árvore de Decisão (CV Estratificado) ===")
print("F1-score por fold:", np.round(f1_scores_tree, 4))
print("Média F1 (CV):      ", np.round(f1_scores_tree.mean(), 4))
print("Desvio Padrão (CV): ", np.round(f1_scores_tree.std(), 4))


=== Árvore de Decisão (CV Estratificado) ===
F1-score por fold: [0.4151 0.4197 0.4227 0.4231 0.4213]
Média F1 (CV):       0.4204
Desvio Padrão (CV):  0.0029


## Avaliação da Árvore de Decisão em CV Estratificado

- **F1-scores por fold:** [0.4151, 0.4197, 0.4227, 0.4231, 0.4213]  
- **Média F1 (CV):** 0.4204  
- **Desvio Padrão (CV):** 0.0029  

### Principais Observações

1. **Desempenho Competitivo**  
   - A Árvore de Decisão alcançou F1 médio ≈ 0.420, muito próximo ao da Regressão Logística (≈ 0.426) e bem acima do KNN (≈ 0.273).  
   - Isso mostra que um modelo não linear simples, com `class_weight='balanced'`, captura interações importantes entre as features.

2. **Variabilidade Moderada**  
   - O desvio padrão (≈ 0.003) é um pouco maior que o da Regressão Logística (≈ 0.002), mas ainda baixo, indicando estabilidade razoável entre as folds.

3. **Complexidade vs. Simplicidade**  
   - Diferentemente da Regressão Logística, a Árvore de Decisão não exige normalização nem engenharia de features polinomiais para obter bom desempenho.  
   - Contudo, corre o risco de overfitting se não for regularizada (poda, profundidade limitada).
## Questao 3 C
### Comparação Rápida


| Modelo                | Média F1 (CV) | Desvio Padrão |
|-----------------------|--------------:|--------------:|
| KNN (K=1)             |        0.2733 |        0.0076 |
| Regressão Logística   |        0.4258 |        0.0020 |
| Árvore de Decisão     |        0.4204 |        0.0029 |

- A **Regressão Logística** lidera em F1 médio e robustez, mas requer normalização.  
- A **Árvore de Decisão** quase empata, sem precisar de pré-processamento complexo, e pode ser beneficiada por poda.  
- O **KNN** fica atrás, evidenciando baixa capacidade de generalização para este problema.

> **Próximo passo (Questão 3C):**  
> Comparar formalmente esses modelos (Logística vs. Árvore de Decisão vs. KNN) no conjunto de teste, apresentando métricas e escolhendo o mais robusto para implantação.  


## Questao 3 D

## Indícios de Overfitting e Underfitting nos Modelos

| Modelo                | Métrica CV (F1 médio) | Desvio Padrão | Pré‐processamento           | Complexidade            |
|-----------------------|----------------------:|--------------:|-----------------------------|-------------------------|
| **KNN (K=1)**         | 0.2733               | 0.0076        | Escalonamento               | Alta (K mínimo)         |
| **Regressão Logística** | 0.4258             | 0.0020        | Escalonamento, balanceamento | Baixa (linear)          |
| **Árvore de Decisão** | 0.4204               | 0.0029        | Balanceamento               | Moderada (sem poda)     |

### KNN (K = 1)
- **Alto viés de variância**: K muito baixo “memoriza” pontos de treino, capturando ruído (overfitting).  
- **Sinais**: desempenho instável entre folds (desvio padrão 0.0076) e fraca generalização (CV F1 ≈ 0.27 vs. teste F1 ≈ 0.29).  
- **Complexidade**: modelo extremamente local, alta variância, baixo viés de treinamento, mas sob alto risco de overfitting.

### Regressão Logística
- **Baixa variância, maior viés**: modelo linear impõe restrições formais (underfitting leve), mas com balanceamento de classes reduz o viés contra a minoritária.  
- **Sinais**: CV F1 alto e extremamente consistente (desvio 0.002), indicando boa generalização e baixo overfitting.  
- **Complexidade**: simples, controlada por `class_weight`, trade-off adequado entre viés e variância.

### Árvore de Decisão
- **Moderada variância e baixo viés**: árvores tendem a sobreajustar se não podadas (underfitting raro, mas possível).  
- **Sinais**: CV F1 (0.4204) quase igual ao da regressão, mas desvio um pouco maior (0.0029), sugerindo leve sensibilidade às amostras de treino.  
- **Complexidade**: flexível, pode capturar não‐linearidades; sem poda, indicada leve tendência a overfitting, mas não severa neste contexto.

---

### Conclusão sobre Viés–Variância
- **Overfitting**: claro no KNN com K=1 (alta variância).  
- **Underfitting**: não visto; mesmo o modelo linear atinge bom F1, sem indicar capacidade excessivamente limitada.  
- **Melhor Equilíbrio**: Regressão Logística—baixa variância e viés controlado via `class_weight`, resultando em performance estável e robusta.  
